In [34]:
import torch
from torch import nn
from tqdm.auto import tqdm
from torchvision import transforms
from torchvision.datasets import MNIST # Training dataset
from torchvision.utils import make_grid
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import math

In [24]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# Discriminator

**paper manuscript about discriminator:**
The discriminator consisted of five convolutional layers followed by an average pooling layer and two fully connected layers. The output layer of the discriminator was a 2-way softmax and the network was trained to discriminate the reconstructed image from the original image.

In [26]:
class japan_discriminator(nn.Module):
    """
    Args:
        - init_features : int
    Inputs:
        - x : Tensor : (N, C, H, W)
    Outputs:
        - : Tensor : (N, 2)
    """
    def __init__(self, init_features = 32):
        super(japan_discriminator, self).__init__()

        self.disc_1 = nn.Sequential(
            japan_discriminator._block_Conv2D(3, init_features, 7, 4),
            japan_discriminator._block_Conv2D(init_features, 2*init_features, 5, 1),
            japan_discriminator._block_Conv2D(2*init_features, 4*init_features, 3, 2),
            japan_discriminator._block_Conv2D(4*init_features, 8*init_features, 3, 1),
            japan_discriminator._block_Conv2D(8*init_features, 8*init_features, 3, 2),
            nn.AvgPool2d(11,11)
        )


        self.disc_2 = nn.Sequential(
            japan_discriminator._block_FC(8*init_features, 8*init_features),
            nn.ReLU(),
            japan_discriminator._block_FC(8*init_features, 2)
        )

    def forward(self, x):
        return self.disc_2(
            self.disc_1(x).flatten(start_dim=1)
        )


    @staticmethod
    def _block_Conv2D(in_channels, out_channels, kernel_size, stride):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size, stride),
            nn.ReLU()
        )

    @staticmethod
    def _block_FC(in_features, out_features):
        return nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(in_features, out_features)
        )

In [27]:
# TEST
N, C, H, W = 64, 3, 227, 227
device = 'cuda'
disc = japan_discriminator().to(device)

x = torch.empty(N, C, H, W).normal_(0,1.0).to(device)

disc(x).shape

torch.Size([64, 2])

In [28]:
# Number of parameters
count_parameters(disc)

1081410

# Generator

**paper manuscript about generator:**
The generator consisted of three fully connected layers followed by six upconvolution layers that generated the final reconstruction image.

In [116]:
class japan_generator(nn.Module):
    """
    Args:
        - num_voxel : int
        - noise_shape : tuple
    Inputs:
        - x : Tensor : (N, num_voxel)
    Outputs:
        - : Tensor : (N, 3, H, W)
    """
    def __init__(self, num_voxel, noise_shape = (64,2,2)):
        super(japan_generator, self).__init__()

        self.noise_shape = noise_shape

        self.gen_1 = nn.Sequential(
            japan_generator._block_FC(num_voxel, math.prod(noise_shape)),
            japan_generator._block_FC(math.prod(noise_shape), math.prod(noise_shape)),
            japan_generator._block_FC(math.prod(noise_shape), math.prod(noise_shape))
        )


        """
        self.gen_2 = nn.Sequential(
            japan_generator._block_UpConv2D(noise_shape[0], noise_shape[0], 4, 2),
            japan_generator._block_UpConv2D(noise_shape[0], 2*noise_shape[0], 3, 1),
            japan_generator._block_UpConv2D(2*noise_shape[0], noise_shape[0], 4, 2),
            japan_generator._block_UpConv2D(noise_shape[0], noise_shape[0], 3, 1),
            japan_generator._block_UpConv2D(noise_shape[0], int(noise_shape[0]/2), 4, 2),
            japan_generator._block_UpConv2D(int(noise_shape[0]/2), int(noise_shape[0]/2), 3, 1),
            japan_generator._block_UpConv2D(int(noise_shape[0]/2), int(noise_shape[0]/4), 4, 2),
            japan_generator._block_UpConv2D(int(noise_shape[0]/4), int(noise_shape[0]/8), 4, 2),
            japan_generator._block_UpConv2D(int(noise_shape[0]/8), 3, 4, 2)
        )
        """


        self.gen_2 = nn.Sequential(
            japan_generator._block_UpConv2D(int(noise_shape[0]/1), int(noise_shape[0]/1), 6, 2),
            japan_generator._block_UpConv2D(int(noise_shape[0]/1), int(noise_shape[0]/2), 6, 2),
            japan_generator._block_UpConv2D(int(noise_shape[0]/2), int(noise_shape[0]/2), 6, 2),
            japan_generator._block_UpConv2D(int(noise_shape[0]/2), int(noise_shape[0]/4), 6, 2),
            japan_generator._block_UpConv2D(int(noise_shape[0]/4), int(noise_shape[0]/8), 6, 2),
            japan_generator._block_UpConv2D(int(noise_shape[0]/8), 3, 6, 2)
        )





    def forward(self, x):
        return self.gen_2(
            self.gen_1(x).view((-1,)+self.noise_shape)
        )


    @staticmethod
    def _block_UpConv2D(in_channels, out_channels, kernel_size, stride, final_layer=False):
        if final_layer == False:
            return nn.Sequential(
                nn.ConvTranspose2d(in_channels, out_channels, kernel_size, stride, padding=1),
                nn.LeakyReLU(negative_slope=0.3)
            )
        else:
            return nn.Sequential(
                nn.ConvTranspose2d(in_channels, out_channels, kernel_size, stride, padding=1)
            )


    @staticmethod
    def _block_FC(in_features, out_features):
        return nn.Sequential(
            nn.Linear(in_features, out_features),
            nn.LeakyReLU(negative_slope=0.3)
        )

In [119]:
# TEST
N, num_voxel = 64, 11760
device = 'cpu'
gen = japan_generator(num_voxel).to(device)

x = torch.empty(N, num_voxel).normal_(0,1.0).to(device)

gen(x).shape

torch.Size([64, 3, 254, 254])

In [120]:
# Number of parameters
count_parameters(gen)

3424507